In [1]:
pip install "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
! pip install -U langchain openai langchain-chroma langchain-experimental

In [3]:
pip install pillow pydantic lxml pillow matplotlib tiktoken open_clip_torch torch

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os 
from unstructured.partition.pdf import partition_pdf

In [5]:
path = "/Users/rafsanmallik/Desktop/lab/kh"
pdf_path = os.path.join(path, "/Users/rafsanmallik/Desktop/lab/kh/Entacid.pdf")

In [6]:

raw_pdf_elements = partition_pdf(
    filename=pdf_path,
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=4000,
    new_after_n_chars=3000,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)

In [7]:
tables = []
texts = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        tables.append(str(element))
    elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
        texts.append(str(element))

In [8]:
from langchain_community.embeddings import FakeEmbeddings

In [22]:
import os
import uuid

import chromadb
import numpy as np
from langchain_chroma import Chroma
from langchain_experimental.open_clip import OpenCLIPEmbeddings
from PIL import Image as _PILImage

# Create chroma
vectorstore = Chroma(
    collection_name="mm_rag_clip_photos", embedding_function=OpenCLIPEmbeddings()
)

# Get image URIs with .jpg extension only
image_uris = sorted(
    [
        os.path.join(path, image_name)
        for image_name in os.listdir(path)
        if image_name.endswith(".jpg")
    ]
)

# Add images
vectorstore.add_images(uris=image_uris)

# Add documents
vectorstore.add_texts(texts=texts)






['af106fab-985a-4912-afe9-9b93a7197d71',
 '95eaf5db-0958-4b31-b586-f3f430a23ef5',
 'a156b82e-e952-4c2f-83bb-f6992f886f39',
 '52faae4d-f7c3-431b-9671-78a3cf85a9e9',
 '9131d4ba-01eb-4669-baea-9ee744520b51',
 'b0525f95-477b-4029-a8b5-2b39f50e65ca',
 'aeacb6df-ec40-4cf2-8dc4-3ad28eb2e01b',
 '7b107971-6185-4e72-8b5d-ce5d5d4e9ee8',
 '62d4e49c-4d27-4ca3-9b2c-d72ec08da255',
 'eb5985e7-4148-4016-9c86-88fdd246586c',
 '9c2366bd-575d-4442-b984-38b42e3e2f63',
 '64f91ac4-1ef1-487c-98de-e701c2b0fdbe',
 '38b52da7-258f-4a7a-8404-0dd73d75ca82',
 'c90f5bbd-e808-42e7-aad8-97aad8f7fae4',
 '4e8f3c40-8e17-46e1-a098-94de2c35f585',
 '365232b6-2001-49f4-83c6-1aa2095f44df',
 '6015fbe3-85c1-4e27-98f9-fea120aefd02',
 'abf4a5a7-87f9-4045-aeac-23aa4889f517',
 '6840aa5e-fc9f-4dd9-ace2-4e44f56dc39b',
 '678e36e0-61ca-4cc9-9e9b-c9755c9c0983',
 '12a99306-17b3-4aeb-adf6-77d1e6e2104f',
 'fd781465-8cc9-4675-8e56-9cd74cc8b3ec',
 'd2912a16-d809-4d3e-b404-aee346e7b108',
 '22dd0dda-04c5-4ae4-8892-26edddaedaa9',
 'ea7a125a-beec-

In [61]:
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "score_threshold":0.05,  # Tweak this value
        "k": 50
    }
)

In [64]:
import base64
import io
from io import BytesIO

import numpy as np
from PIL import Image


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string.

    Args:
    base64_string (str): Base64 string of the original image.
    size (tuple): Desired size of the image as (width, height).

    Returns:
    str: Base64 string of the resized image.
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def is_base64(s):
    """Check if a string is Base64 encoded"""
    try:
        return base64.b64encode(base64.b64decode(s)) == s.encode()
    except Exception:
        return False


def split_image_text_types(docs):
    """Split numpy array images and texts"""
    images = []
    text = []
    for doc in docs:
        doc = doc.page_content  # Extract Document contents
        if is_base64(doc):
            # Resize image to avoid OAI server error
            images.append(
                resize_base64_image(doc, size=(250, 250))
            )  # base64 encoded str
        else:
            text.append(doc)
    return {"images": images, "texts": text}

In [62]:
from langchain_ollama import OllamaLLM
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def prompt_func(data_dict):
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

  
    if data_dict["context"]["images"]:
        image_message = {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{data_dict['context']['images'][0]}"
            },
        }
        messages.append(image_message)


    text_message = {
        "type": "text",
        "text": (
            "As an expert pharmacist, your task is to analyze and interpret images of medicines and prescriptions and suggest medicines and help patients and doctors when they ask you questions ...\n\n"
            f"User-provided keywords: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)

    return [HumanMessage(content=messages)]


model = OllamaLLM(model="gemma3:4b", temperature=0.1, max_tokens=2000)  


chain = (
    {
        "context": retriever | RunnableLambda(split_image_text_types),
        "question": RunnablePassthrough(),
    }
    | RunnableLambda(prompt_func)
    | model
    | StrOutputParser()
)

In [67]:
response=chain.invoke("should i CONTINUE NAPA for fever?")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='deb1500c-c3c2-4882-b65f-f5d8b6f7766d', metadata={}, page_content="milk. Exercise caution when administering to a nursing mother. Precautions & Warnings For external use only. Keep away from eyes, nose, mouth and other mucous membrane. Use of antibiotics (especially prolonged or repeated therapy) may result in bacterial or fungal overgrowth of non-susceptible organisms. Such overgrowth may lead to a secondary infection. Take appropriate measures if superinfections occur. Use in Special Populations Safety and effectiveness in children less than 12 years have not been established. Therapeutic Class Topical antibiotics for Acne Storage Conditions Keep below 25°C temperature, away from light & moisture. Keep out of the reach of children. Pack Image: Eromycin 3% Lotion\n\nAcorex Syrup Pack Image Amb

In [68]:
print(response)

Okay, let’s analyze the information provided to address the user’s question: “Should I CONTINUE NAPA for fever?”

**Here’s my assessment based on the provided text:**

*   **The User’s Query is Misinterpreted:** The user is asking about continuing *NAPA* for a fever. However, the text primarily discusses *Ambroxol Hydrochloride* (in Acorex syrup) and *Eromycin 3% Lotion*. There is no mention of "NAPA" in the provided documentation.

*   **Ambroxol Hydrochloride (Acorex) is Relevant for Fever Symptoms:** Acorex, containing Ambroxol Hydrochloride, *is* indicated for productive cough, which is a common symptom of fever and respiratory infections. It’s a mucolytic agent designed to help loosen and expel mucus, which can be beneficial when a fever is accompanied by a cough.

*   **Important Considerations & Warnings:**
    *   **Don’t Combine with Antitussives:** The text explicitly warns against taking Acorex simultaneously with antitussives like Codeine.  If phlegm is liquefied by Acorex,

In [110]:
from IPython.display import HTML, display


def plt_img_base64(img_base64):
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    # Display the image by rendering the HTML
    display(HTML(image_html))


docs = retriever.invoke("Entacyd ", k=1)
for doc in docs:
    if is_base64(doc.page_content):
        plt_img_base64(doc.page_content)
    else:
        print(doc.page_content)

Entacyd Plus Chewable Tablet Pack Image Aluminium Hydroxide + Magnesium Hydroxide + Simethicone 400 mg+400 mg+30 mg Square Pharmaceuticals PLC Unit Price: ৳ 2.50 (20 x 10: ৳ 500.00) Strip Price: ৳ 25.00 Also available as: (200 mg+400 mg+30 mg)/5 ml (Suspension) Alternate Brands Indications This is indicated for symptomatic relief of hyperacidity associated with the peptic ulcer, gastritis, peptic oesophagitis, gastric hyperacidity, heartburn, sour stomach or hiatus hernia. It is effective in the prevention of stress ulceration and GI bleeding. It acts as an antiflatulent to alleviate the symptoms of gas including post operative gas pain. This rapidly relieves acid pain, disperses gastric foam and facilitates eructation of gas and air. * রিজস্টাড িচিকৎসেকর পরামশ মাতােবক ঔষধ সবন করুন' Composition Each tablet contains- Dried Aluminium Hydroxide Gel BP (Al2O3, 47% minimum): 425.53 mg Magnesium Hydroxide BP: 400 mg Simethicone USP: 30 mg

Simethicone USP: 30 mg Each 5 ml contains- Aluminium